In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn

In [5]:
a = torch.ones((512, 1024))

x = a.split(2, dim=-1)

In [ ]:
non_kv = torch.load("k.tensor")
kv = torch.load("k_kv.tensor")

In [ ]:
torch.abs(kv - non_kv).max()


In [ ]:
non_kv_l5 = torch.load("k_layer5.tensor")
kv_l5 = torch.load("k_kv_layer5.tensor")

In [3]:
layers = 6

for i in range(layers):
    kv = torch.load(f"out_kv_layer{i}.tensor", weights_only=True)
    non_kv = torch.load(f"out_layer{i}.tensor", weights_only=True)
    print(f"diff at layer {i}: {torch.abs(kv - non_kv).max()}")


diff at layer 0: 1.5497207641601562e-06
diff at layer 1: 0.5641153454780579
diff at layer 2: 0.8714048862457275
diff at layer 3: 0.8938214182853699
diff at layer 4: 1.0913362503051758
diff at layer 5: 1.0506775379180908


In [2]:
layers = 6

for i in range(layers):
    kv = torch.load(f"k_kv_layer{i}.tensor")
    non_kv = torch.load(f"k_layer{i}.tensor")
    print(f"diff at layer {i}: {torch.abs(kv - non_kv).max()}")


diff at layer 0: 1.7285346984863281e-06
diff at layer 1: 0.514729380607605
diff at layer 2: 0.8702945709228516
diff at layer 3: 0.8971630334854126
diff at layer 4: 1.091021180152893
diff at layer 5: 1.0824984312057495


/var/folders/fw/tk4xwnt92g588pqyvts_spwr0000gn/T/ipykernel_15908/1422871628.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  kv = torch.load(f"k_kv_layer{i}.tensor")
/var

In [ ]:
torch.allclose(kv[0], non_kv[0], atol=1e-6)

In [ ]:
kv[:, :-1].shape

In [ ]:
kv[0, 0, 5, 0], non_kv[0, 0, 5, 0]

In [ ]:
from omni.preprocessing.tokenizer import AutoTokenizer

tokenizer = AutoTokenizer.create("EleutherAI/gpt-neo-125m")
tokenizer.add_special_tokens({"pad_token": "<pad>"})

In [ ]:
import torch

from omni.architectures.llama import LlamaConfig
from omni.modules.transformer import Transformer
from omni.preprocessing.tokenizer import AutoTokenizer
from omni.utils.system import auto_device

tokenizer = AutoTokenizer.create("EleutherAI/gpt-neo-125m")
tokenizer.add_special_tokens({"pad_token": "<pad>"})

llama_config = LlamaConfig(
    vocab_size=50258,
    seq_len=512,
    d_model=256,
    num_heads=8,
    num_kv_heads=8,
    num_layers=4,
    rope_theta=0.1,
    norm_eps=1e-6,
    activation_fn="silu",
    mlp_bias=False,
    mlp_dropout=0.0,
    attention_bias=False,
    attention_dropout=0.0,
    pos_encoding_type="rope",
    mlp="mlp_swiglu",
    normalization="rmsnorm",
    attention="gqa",
)

model = Transformer(llama_config)

## create KV cache

checkpoint = torch.load("checkpoints/llama-30M_20250123_104138/init.ckpt", weights_only=True)
model.load_state_dict(checkpoint["model"])

device="mps"
model = model.to(device)
model.eval()

In [5]:
device="mps"
prompt = "Once upon a time"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

In [6]:
outputs = model(input_ids)
next_token_logits = outputs[:, -1, :]

In [ ]:
top_k_values, top_k_indices = torch.topk(logits, self.top_k, dim=-1)

In [ ]:
top_k_values = torch.tensor([float("-inf"), 1.0, 2.0])

top_k_values
nn.functional.softmax(top_k_values, dim=-1)

In [ ]:
float("-inf")

In [ ]:
sorted_logits, sorted_indices = torch.sort(
    top_k_values, dim=-1, descending=True
)
cumulative_probs = torch.cumsum(
    nn.functional.softmax(sorted_logits, dim=-1), dim=-1
)

In [ ]:
next_token_logits

In [ ]:
visualize_logit_distribution(next_token_logits[0], top_k=500)

In [18]:
import torch
import matplotlib.pyplot as plt

def visualize_logit_distribution(logits, top_k=500):
    """
    Visualizes the logit distribution by focusing on the top-k logits.

    Args:
        logits (torch.Tensor): Logits tensor of shape (vocab_size,).
        top_k (int): Number of top logits to visualize. Defaults to 500.
    """
    if logits.dim() != 1:
        raise ValueError("Logits tensor must be 1-dimensional (vocab_size,).")

    # Convert logits to probabilities and take log
    log_probs = torch.log_softmax(logits, dim=0)

    # Get the top-k log probabilities and their indices
    top_log_probs, top_indices = torch.topk(log_probs, k=top_k)

    # Sort the top-k log probabilities for visualization
    sorted_log_probs, sorted_indices = torch.sort(top_log_probs, descending=True)

    # Plot the distribution
    plt.figure(figsize=(10, 6))
    plt.plot(sorted_log_probs.detach().cpu().numpy(), marker="o", linestyle="-")
    plt.title(f"Log Probability Distribution (Top {top_k})", fontsize=14)
    plt.xlabel("Rank", fontsize=12)
    plt.ylabel("Log Probability Value", fontsize=12)
    plt.grid(alpha=0.5)
    plt.show()


In [13]:
import tokenizers.processors as processors

def _add_bos_token(tokenizer: AutoTokenizer):
    tokenizer._tokenizer.post_processor = processors.Sequence(
        [
            processors.ByteLevel(trim_offsets=False),
            processors.TemplateProcessing(
                single=f"{tokenizer.bos_token}:0 $A:0",
                pair=f"{tokenizer.bos_token}:0 $A:0 {tokenizer.bos_token}:1 $B:1",
                special_tokens=[
                    (tokenizer.bos_token, tokenizer.bos_token_id),
                ],
            ),
        ]
    )
    return tokenizer

In [14]:
tokenizer2 = _add_bos_token(tokenizer) 

In [ ]:
torch.log(torch.tensor(50000))

In [16]:
test = "thjis is a test, this is a test"

output = tokenizer2(test, padding="max_length", max_length=3, truncation=True, return_overflowing_tokens=True)

In [ ]:
d_model=128
hidden_dim = 4 * int(2 * d_model / 3)
hidden_dim = 4 * (
    (hidden_dim + 4 - 1) // 4
)

hidden_dim

In [ ]:
import torch

logits = torch.tensor([1,5,10], dtype=torch.float32)
torch.softmax(logits, dim=0)

In [ ]:
tokenizer.special_tokens_map

In [ ]:
tokenizer.encode(x, padding="max_length", max_length=max_length, truncation=True, return_overflowing_tokens=True)

In [ ]:
tokenizer(test, )

In [ ]:
output

In [ ]:
from omni.modules.pos_embeddings import precompute_freqs_cis_real

pos_embeddings = precompute_freqs_cis_real(64, 512)

In [ ]:
base = torch.pow(2.0, torch.tensor(-8/16))

In [7]:
def _tokenize(dataset, tokenizer: AutoTokenizer, num_proc):
    print("Tokenizing dataset...")
    dataset = dataset.map(
        lambda x: tokenizer(
            x["text"],
            truncation=True,
            max_length=512,
            padding="max_length",
            return_overflowing_tokens=True,
        ),
        batched=True,
        num_proc=num_proc,
        remove_columns=dataset.column_names,
    )
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
    return dataset

In [4]:
from datasets import load_dataset

ds = load_dataset("roneneldan/TinyStories", split="train", num_proc=1)

In [ ]:
_tokenize(ds, tokenizer, 2)